### WeRateDogs Data: Gathering and Wrangling
##### WeRateDogs twitter account posts images of dogs along with a description and a rating for the dogs.
#### I have followed 4 step process:
#### 1.GATHER
#### 2.ASSESS
#### 3.CLEAN
#### 4.ANALYSE(Refer to act_report.ipynb)




### Step-1-GATHER
###### Data Gathering is done from 3 sources:
######            1.csv file(twitter-archive-enhanced.csv)
######            2.tsv file(image_predictions.tsv')
######            3.Twitter’s API tweepy

In [1]:
# Import Libraries
import requests
import tweepy
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns
sns.set()

## The WeRateDogs Twitter archive 
###### csv file(twitter-archive-enhanced.csv)

In [2]:
df_archive=pd.read_csv(r'C:\Users\swaindi\OneDrive - Hewlett Packard Enterprise\2021\Knowledge\Udacity Data Wrangling\Project\twitter-archive-enhanced.csv')
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

**The file twitter-archive-enhanced.csv successfully stored in archive data frame, it has 17 columns and 2356 entries**

In [3]:
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
# Making a list of tweet_id
tweet_ids = df_archive.tweet_id

## Downloading the Tweet Image Predictions Programmatically
##### tsv file(image_predictions.tsv')

###  Remarks:Since the below code didnot work due to issue with the proxy, the tsv file was directly executed based on the input from the technical mentor
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
resp = requests.get(url)
with open('data/' + url.split('/')[-1], mode='wb') as file:
    file.write(resp.content)

In [5]:
image_prediction = pd.read_csv(r'C:\Users\swaindi\OneDrive - Hewlett Packard Enterprise\2021\Knowledge\Udacity Data Wrangling\Final_Data_Wrangling\image-predictions (2).tsv', sep='\t')
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [6]:
image_prediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## Download from twitter API tweety for additional tweet
#### Twitter's API Tweepy

In [7]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys

import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO REVIEWER: I had mobile verification issues so the following
# Twitter API code was sent to me from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_archive.tweet_id.values
len(tweet_ids)

2356

In [8]:
#### Save each tweet's returned JSON as a new line in a .txt file
#with open(r'C:\Users\swaindi\OneDrive - Hewlett Packard Enterprise\2021\Knowledge\Udacity Data Wrangling\Project\tweet-json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
   # for tweet_id in tweet_ids:
    #    count += 1
     #   print(str(count) + ": " + str(tweet_id))
      #  try:
     #       tweet = api.get_status(tweet_id, tweet_mode='extended')
       #     print("Success")
        #    json.dump(tweet._json, outfile)
        #    outfile.write('\n')
       # except tweepy.TweepError as e:
        #    print("Fail")
        #    fails_dict[tweet_id] = e
         #   pass
#end = timer()
#print(end - start)
#print(fails_dict)

## Due to authentication failure I have directly retrieved the tweet-json.txt file

In [9]:
tweet_list=[]
with open(r'C:\Users\swaindi\OneDrive - Hewlett Packard Enterprise\2021\Knowledge\Udacity Data Wrangling\Project\tweet-json (1) - Copy.txt','r') as f:
    for row in f:
        json_dict= json.loads(row)
        to_append= {
            'tweet_id':json_dict['id_str'],
            'retweet_count':json_dict['retweet_count'],
            'favorite_count':json_dict['favorite_count']
        }
        tweet_list.append(to_append)

In [10]:
df_json=pd.DataFrame(tweet_list, columns=['tweet_id','retweet_count','favorite_count'])


In [11]:
df_json.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   retweet_count   2354 non-null   int64 
 2   favorite_count  2354 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 55.3+ KB


In [12]:
df_json['tweet_id'] = df_archive['tweet_id'].astype(int)
df_json['tweet_id']=df_json['tweet_id'].abs()
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2354 non-null   int32
 1   retweet_count   2354 non-null   int64
 2   favorite_count  2354 non-null   int64
dtypes: int32(1), int64(2)
memory usage: 46.1 KB


In [13]:
df_json.head(5)

,tweet_id,retweet_count,favorite_count
0,648585215,8853,39467
1,1100394498,6514,33819
2,1286119424,4328,25461
3,1885904896,8964,42908
4,1079468032,9774,41048


## ASSESS
### Create a master file and assess the attributes

In [14]:
# Since we want to create one high quality and tidy master pandas DataFrame
# we will start by merging our dataframe in one
df_master = pd.merge(df_archive, image_prediction, how = 'left', on = ['tweet_id'] )
df_master = pd.merge(df_master, df_json, how = 'left', on = ['tweet_id'])
df_master.to_csv('df_master.csv', encoding = 'utf-8')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [15]:
df_master.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,0.097049,False,bagel,0.085851,False,banana,0.076110,False,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,...,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,...,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,...,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,NaN,NaN


In [16]:
## Tweets starting with RT..Count the manual tweets
sum(df_master.text.str.startswith("RT"))

183

In [17]:
## Missing expanded_url
sum(df_master.expanded_urls.isnull())

59

In [18]:
## Check for nan values in in_reply_to_status_id
df_master['in_reply_to_status_id'].isna().sum()

2278

## Assessed and identified issues

###  Quality Issues Identified:<p>
1.retweeted_status_timestamp, timestamp should be datetime instead of object (string)<p>
2.in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id should be 
   object instead of float.We don't want any operations on them images dataset.<p>
3."rating_numerator" and "rating denominator" columns are two columns for data point: rating.We can 
  calculate the rating and drop these columns<p>     
4.Texts of tweets that start with "RT" are manual tweets and should be removed from dataset..<p>
5.Missing "expanded_urls" values in 59 rows: meaning no image with tweet and those should be 
  removed'<p>  
6.Few column names like p1,p2,p3,p4 are unclear.Replace by more descriptive column name<p>
7.In_reply_to_status_id has missing data.<p>
8.Image predictions in columns `p1`, `p2` and `p3` have underscores.<p>



###  Tidiness Issues:<p>
9. Columns: ['Doggo', 'floofer', 'pupper', 'puppo'] can be condensed to one column..<p>
10.Remove the "not-needed" columns from the Master tables.>

## 3.CLEAN
#### Resolving the quality issues

##### 1.retweeted_status_timestamp, timestamp should be datetime instead of object (string)

In [19]:
from datetime import datetime as dt
df_master.timestamp = pd.to_datetime(df_master['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
df_master.retweeted_status_timestamp = pd.to_datetime(df_master['retweeted_status_timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   int64              
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

**2.in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id should be object instead of float.We don't want any operations on them images dataset.**

In [20]:
df_master['in_reply_to_status_id'] = df_master['in_reply_to_status_id'].astype(str)
df_master['in_reply_to_user_id'] = df_master['in_reply_to_user_id'].astype(str)
df_master['retweeted_status_id'] = df_master['retweeted_status_id'].astype(str)
df_master['retweeted_status_user_id'] = df_master['retweeted_status_user_id'].astype(str)

In [21]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   int64              
 1   in_reply_to_status_id       2356 non-null   object             
 2   in_reply_to_user_id         2356 non-null   object             
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         2356 non-null   object             
 7   retweeted_status_user_id    2356 non-null   object             
 8   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

**3."rating_numerator" and "rating denominator" columns are two columns for data point: rating.We can calculate the rating and drop these columns**

In [22]:
df_master['rating']=df_master['rating_numerator']
##df_master.head.drop(df_master[['rating_numerator ','rating_denominator']],axis=1,inplace=True)
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   int64              
 1   in_reply_to_status_id       2356 non-null   object             
 2   in_reply_to_user_id         2356 non-null   object             
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         2356 non-null   object             
 7   retweeted_status_user_id    2356 non-null   object             
 8   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

**4.Texts of tweets that start with "RT" are manual tweets and should be removed from dataset.**

In [23]:
##Count the manual tweets
sum(df_master.text.str.startswith("RT"))

183

In [24]:
## Delete the manual re-tweets
df_master.drop(df_master[df_master.text.str.startswith("RT")].index, inplace=True)
##Check
sum(df_master.text.str.startswith("RT"))

0

**5.Missing "expanded_urls" values in 58 rows: meaning no image with tweet and those should be removed'**

In [25]:
sum(df_master.expanded_urls.isnull())

58

In [26]:
df_master.dropna(subset=['expanded_urls'], inplace=True)

**6.Few column names are unclear.Replace by more descriptive column name**

In [27]:
df_master.rename(columns={"img_num":"number_of_images", "p1":"prediction_1", "p1_conf":"p1_confidence", "p2":"prediction_2", "p2_conf":"p2_confidence", "p3":"prediction_3", "p3_conf":"p3_confidence"}, inplace=True)

**7.In_reply_to_status_id has missing data.Replace NA**

In [28]:
df_master.dropna(subset=['in_reply_to_status_id'], inplace=True)

**8.The Prediction columns  p3 have underscores.Replace them**

In [29]:
df_master["prediction_1"] = df_master["prediction_1"].str.replace("_", " ")
df_master["prediction_2"] = df_master["prediction_2"].str.replace("_", " ")
df_master["prediction_3"] = df_master["prediction_3"].str.replace("_", " ")

**9.Columns: ['Doggo', 'floofer', 'pupper', 'puppo'] can be condensed to one column.**

In [30]:
df_master["doggo"] = df_master.doggo.str.replace('None', '')
df_master["floofer"] = df_master.floofer.str.replace('None', '')
df_master["pupper"] = df_master.pupper.str.replace('None', '')
df_master["puppo"] = df_master.puppo.str.replace('None', '')

In [31]:
df_master["dog_type"] = df_master.doggo + df_master.floofer + df_master.pupper + df_master.puppo
df_master.dog_type.value_counts()

                1777
pupper           222
doggo             72
puppo             23
doggopupper       10
floofer            9
doggopuppo         1
doggofloofer       1
Name: dog_type, dtype: int64

In [32]:
df_master.drop(["doggo", "floofer", "pupper", "puppo"], axis=1, inplace=True)

**10.Check the columns which are not required and delete them**

In [33]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2115 entries, 0 to 2355
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2115 non-null   int64              
 1   in_reply_to_status_id       2115 non-null   object             
 2   in_reply_to_user_id         2115 non-null   object             
 3   timestamp                   2115 non-null   datetime64[ns, UTC]
 4   source                      2115 non-null   object             
 5   text                        2115 non-null   object             
 6   retweeted_status_id         2115 non-null   object             
 7   retweeted_status_user_id    2115 non-null   object             
 8   retweeted_status_timestamp  0 non-null      datetime64[ns, UTC]
 9   expanded_urls               2115 non-null   object             
 10  rating_numerator            2115 non-null   int64           

In [34]:
df_master = df_master.drop(df_master[['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id' ]], axis=1)
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2115 entries, 0 to 2355
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2115 non-null   int64              
 1   timestamp                   2115 non-null   datetime64[ns, UTC]
 2   source                      2115 non-null   object             
 3   text                        2115 non-null   object             
 4   retweeted_status_timestamp  0 non-null      datetime64[ns, UTC]
 5   expanded_urls               2115 non-null   object             
 6   rating_numerator            2115 non-null   int64              
 7   rating_denominator          2115 non-null   int64              
 8   name                        2115 non-null   object             
 9   jpg_url                     1993 non-null   object             
 10  number_of_images            1993 non-null   float64         

In [35]:
twitter_archive_enhanced=df_master
twitter_archive_enhanced.to_csv('twitter_archive_enhanced.csv',index=False, index_label = True, encoding='utf-8')